In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from os import listdir, makedirs
from os.path import join, exists, expanduser

from sklearn.metrics import log_loss, accuracy_score
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.applications import xception
from keras.applications import inception_v3
from keras.applications.vgg16 import preprocess_input, decode_predictions
from keras.layers import Flatten
from keras.models import Model
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
################
INPUT_SIZE = 224
################
NUM_CLASSES = 120
SEED = 1987
POOLING = 'avg'
data_dir = './data'

## Data

In [3]:
labels = pd.read_csv(join(data_dir, 'labels.csv'))
print(len(listdir(join(data_dir, 'train'))), len(labels))

10222 10222


In [4]:
selected_breed_list = list(labels.groupby('breed').count().sort_values(by='id', ascending=False).head(NUM_CLASSES).index)
labels = labels[labels['breed'].isin(selected_breed_list)]

In [5]:
def read_img(img_id, train_or_test, size):
    """Read and resize image.
    # Arguments
        img_id: string
        train_or_test: string 'train' or 'test'.
        size: resize the original image.
    # Returns
        Image as numpy array.
    """
    img = image.load_img(join(data_dir, train_or_test, '%s.jpg' % img_id), target_size=size)
    img = image.img_to_array(img)
    return img

## Features

In [6]:
x_train_dev = np.zeros((len(labels), INPUT_SIZE, INPUT_SIZE, 3), dtype='float32')
for i, img_id in enumerate(labels['id'][0:3000]):
    img = read_img(img_id, 'train', (INPUT_SIZE, INPUT_SIZE))
    ########################################################
    x = preprocess_input(np.expand_dims(img.copy(), axis=0))
    ########################################################
    x_train_dev[i] = x
    
    if (i % 500 == 0):
        print (str(i), 'images processed')
        
print('Train Images shape: {} size: {:,}'.format(x_train_dev.shape, x_train_dev.size))

0 images processed
500 images processed
1000 images processed
1500 images processed
2000 images processed
2500 images processed
Train Images shape: (10222, 224, 224, 3) size: 1,538,697,216


In [ ]:
x_train, x_dev, y_train, y_dev = train_test_split(x_train_dev, pd.get_dummies(labels['breed']), test_size=0.2, random_state=0)

In [8]:
np.save('./data/bottleneck_features/pretrained_keras_models/x_train.npy', x_train)
np.save('./data/bottleneck_features/pretrained_keras_models/x_dev.npy', x_dev)
np.save('./data/bottleneck_features/pretrained_keras_models/y_train.npy', y_train)
np.save('./data/bottleneck_features/pretrained_keras_models/y_dev.npy', y_dev)

## Model

In [3]:
base_model = VGG16(
    weights='imagenet', 
    input_shape=(INPUT_SIZE, INPUT_SIZE, 3),
    include_top=False, 
    ###############
    pooling=POOLING
    ###############
)

#output = Flatten()(base_model.output)
#model = Model(inputs=base_model.inputs, outputs=output)
model = base_model

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [5]:
x_train = np.load('./data/bottleneck_features/pretrained_keras_models/x_train.npy')
train_vgg_bf = model.predict(x_train, batch_size=8, verbose=1)
np.save('./data/bottleneck_features/pretrained_keras_models/train_vgg_bf.npy', train_vgg_bf)

1421/1421 [==============================] - 679s 478ms/step


In [5]:
x_dev = np.load('./data/bottleneck_features/pretrained_keras_models/x_dev.npy')
dev_vgg_bf = model.predict(x_dev, batch_size=8, verbose=1)
np.save('./data/bottleneck_features/pretrained_keras_models/dev_vgg_bf.npy', dev_vgg_bf)

356/356 [==============================] - 162s 455ms/step


## LogReg

In [6]:
train_vgg_bf = np.load('./data/bottleneck_features/pretrained_keras_models/train_vgg_bf.npy')
dev_vgg_bf = np.load('./data/bottleneck_features/pretrained_keras_models/dev_vgg_bf.npy')
y_train = np.load('./data/bottleneck_features/pretrained_keras_models/y_train.npy')
y_dev = np.load('./data/bottleneck_features/pretrained_keras_models/y_dev.npy')

#train_vgg_bf = np.load('./data/bottleneck_features/vgg16/bottleneck_features_train.npy')
#dev_vgg_bf = np.load('./data/bottleneck_features/vgg16/bottleneck_features_dev.npy')
#y_train = np.load('./data/bottleneck_features/vgg16/train_labels.npy')
#y_dev = np.load('./data/bottleneck_features/vgg16/dev_labels.npy')

In [7]:
logreg = LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=SEED)
logreg.fit(train_vgg_bf, (y_train * range(NUM_CLASSES)).sum(axis=1))
dev_probs = logreg.predict_proba(dev_vgg_bf)
dev_preds = logreg.predict(dev_vgg_bf)

In [8]:
print('Validation VGG LogLoss {}'.format(log_loss(y_dev, dev_probs)))
print('Validation VGG Accuracy {}'.format(accuracy_score((y_dev * range(NUM_CLASSES)).sum(axis=1), dev_preds)))

Validation VGG LogLoss 0.32802901648240856
Validation VGG Accuracy 0.9129213483146067
